## Testing of the Graph Environment

Imports

In [1]:
import numpy as np
import pandas as pd
import json
import os
import shutil
import sys
import gym
from datetime import datetime

import ray

import warnings
warnings.filterwarnings('ignore')


In [2]:
#from ray.rllib.agents.dqn import DQNTrainer, DEFAULT_CONFIG, APEX_DEFAULT_CONFIG
from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG

Import the Environment

In [3]:
sys.path.insert(0,"")

from ManhattanGraph import ManhattanGraph
from gym_graphenv.envs.GraphworldManhattan import GraphEnv

In [4]:
env_config = dict({'pickup_hub_index':2,
        'delivery_hub_index':3,
        'pickup_timestamp': datetime(2016, 6, 14, 22, 0, 0),
        'delivery_timestamp': datetime(2016, 6, 14, 1, 0, 0) 
})

In [5]:
env=GraphEnv()


Run random steps without Ray Trainer

In [6]:
def run_one_episode (env, env_config):
    env.reset()
    print("reset done")
    sum_reward = 0
    for i in range(30):
        action = env.action_space.sample()
        #print(action)
        state, reward, done, info = env.step(action)
        sum_reward+=reward
        #env.render()
        if done:
            print("DELIVERY DONE! sum_reward: ",sum_reward)
            break

        print("sum_reward: ",sum_reward)
        # print("sum_reward: ",sum_reward, " time: ",env.time, "deadline time: ", env.deadline, "pickup time: ", env.pickup_time)
    return sum_reward

for i in range(1): 
    run_one_episode (env, env_config)

None
Reset without config
final hub: 49
current position: 69
reset done
action ==  30  New Position 30
69 -> 30 60.0
Step() Execution time: 0.4843430519104004 seconds
sum_reward:  -60.0
action ==  24  New Position 24
30 -> 24 60.0
Step() Execution time: 0.23652291297912598 seconds
sum_reward:  -120.0
action ==  39  New Position 39
24 -> 39 60.0
Step() Execution time: 0.27385807037353516 seconds
sum_reward:  -180.0
action ==  1  New Position 1
39 -> 1 60.0
Step() Execution time: 0.2523479461669922 seconds
sum_reward:  -240.0
action ==  46  New Position 46
1 -> 46 60.0
Step() Execution time: 0.26940488815307617 seconds
sum_reward:  -300.0
action ==  8  New Position 8
46 -> 8 60.0
Step() Execution time: 0.2369699478149414 seconds
sum_reward:  -360.0
action ==  43  New Position 43
8 -> 43 60.0
Step() Execution time: 0.2483978271484375 seconds
sum_reward:  -420.0
action ==  58  New Position 58
43 -> 58 60.0
Step() Execution time: 0.24225592613220215 seconds
sum_reward:  -480.0
action ==  60

Initialize Ray

In [7]:
ray.init()

2022-05-11 22:44:53,928	INFO services.py:1412 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': None,
 'object_store_address': '/tmp/ray/session_2022-05-11_22-44-49_438600_30014/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-05-11_22-44-49_438600_30014/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-05-11_22-44-49_438600_30014',
 'metrics_export_port': 64882,
 'gcs_address': '127.0.0.1:64893',
 'address': '127.0.0.1:64893',
 'node_id': '5e87440873e4d6de60eac624be0f22f532a94ced45c5afa8df8e7839'}

Set DQN trainer configuration

In [8]:
dqn_config = DEFAULT_CONFIG.copy()
dqn_config['num_workers'] = 1
dqn_config["train_batch_size"] = 400
dqn_config["gamma"] = 0.95
dqn_config["n_step"] = 10
dqn_config["framework"] = "torch"
#num_gpus and other gpu parameters in order to train with gpu
#dqn_config["num_gpus"] = int(os.environ.get("RLLIB_NUM_GPUS", "0")) 

Set DQN - Rainbow configuration

In [ ]:
rainbow_config = DEFAULT_CONFIG.copy()

rainbow_config = DEFAULT_CONFIG.copy()
rainbow_config['num_workers'] = 1
rainbow_config["train_batch_size"] = 400
rainbow_config["gamma"] = 0.95
rainbow_config["n_step"] = 10
rainbow_config["framework"] = "torch"
#num_gpus and other gpu parameters in order to train with gpu
#rainbow_config["num_gpus"] = int(os.environ.get("RLLIB_NUM_GPUS", "0")) 

#rainbow parameters

# N-step Q learning
rainbow_config["n_step"]= 5 #[between 1 and 10]
# Whether to use noisy network
rainbow_config["noisy"] = True
# Number of atoms for representing the distribution of return. When
# this is greater than 1, distributional Q-learning is used.
# the discrete supports are bounded by v_min and v_max
rainbow_config["num_atoms"] =2 #[more than 1]
rainbow_config["v_min"] =-100
rainbow_config["v_max"]=100 # (set v_min and v_max according to your expected range of returns).

Set DQN - Ape-X configuration 

In [ ]:
apex_config=APEX_DEFAULT_CONFIG.copy()

NameError: name 'APEX_DEFAULT_CONFIG' is not defined

Initialize the DQN trainer, Rainbow DQN and DQN Ape-X

In [9]:
ppo_trainer=PPOTrainer(DEFAULT_CONFIG,GraphEnv)

2022-05-11 22:44:59,296	INFO trainer.py:2140 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2022-05-11 22:44:59,300	INFO ppo.py:249 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-05-11 22:44:59,302	INFO trainer.py:779 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=30057) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
(RolloutWorker pid=30056) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient

(RolloutWorker pid=30056) {}
(RolloutWorker pid=30056) Reset without config
(RolloutWorker pid=30056) final hub: 39
(RolloutWorker pid=30056) current position: 20
(RolloutWorker pid=30057) {}
(RolloutWorker pid=30057) Reset without config
(RolloutWorker pid=30057) final hub: 50
(RolloutWorker pid=30057) current position: 21


2022-05-11 22:45:20,490	INFO trainable.py:127 -- Trainable.setup took 21.198 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-05-11 22:45:20,495	WARNING util.py:55 -- Install gputil for GPU system monitoring.


In [ ]:
trainer_dqn = DQNTrainer(dqn_config,GraphEnv)

In [ ]:
trainer_rainbow=DQNTrainer(rainbow_config,GraphEnv )


In [ ]:
#trainer_apex=DQNTrainer(apex_config,GraphEnv )

Define the path where the results of the trainer should be saved

In [ ]:
checkpoint_root = "tmp/dqn/graphworld"
shutil.rmtree(checkpoint_root, ignore_errors=True, onerror=None)   # clean up old runs

Run the trainer

In [20]:
results = []
episode_data = []
episode_json = []
n_iter=10

for n in range(n_iter):
    result = ppo_trainer.train()
    results.append(result)
    print("TEST",n)
    
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'], 
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']
              }
    
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = ppo_trainer.save(checkpoint_root)
    
    print(f'{n+1:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}, len mean: {result["episode_len_mean"]:8.4f}. Checkpoint saved to {file_name}')

(RolloutWorker pid=28020) {}
(RolloutWorker pid=28020) Reset without config
(RolloutWorker pid=28020) final hub: 24
(RolloutWorker pid=28020) current position: 49
(RolloutWorker pid=28019) {}
(RolloutWorker pid=28019) Reset without config
(RolloutWorker pid=28019) final hub: 17
(RolloutWorker pid=28019) current position: 19
(RolloutWorker pid=28020) Does space contain state: True
(RolloutWorker pid=28020) State: {'cost': array([60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
(RolloutWorker pid=28020)        60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
(RolloutWorker pid=28020)        60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 20, 60,
(RolloutWorker pid=28020)        60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
(RolloutWorker pid=28020)        60, 60]), 'current_hub': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
(RolloutWorker pid=28020)        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

(RolloutWorker pid=28019) FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
(RolloutWorker pid=28020) FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.


(RolloutWorker pid=28020) action ==  22  New Position 22
(RolloutWorker pid=28019) action ==  2  New Position 2
(RolloutWorker pid=28019) 19 -> 2 60.0
(RolloutWorker pid=28019) Step() Execution time: 0.5871288776397705 seconds
(RolloutWorker pid=28019) action ==  39  New Position 39
(RolloutWorker pid=28020) 49 -> 22 60.0
(RolloutWorker pid=28020) Step() Execution time: 0.5893559455871582 seconds
(RolloutWorker pid=28020) action ==  42  New Position 42
(RolloutWorker pid=28020) 22 -> 42 60.0
(RolloutWorker pid=28020) Step() Execution time: 0.3338329792022705 seconds
(RolloutWorker pid=28020) action ==  36  New Position 36
(RolloutWorker pid=28019) 2 -> 39 60.0
(RolloutWorker pid=28019) Step() Execution time: 0.34353089332580566 seconds
(RolloutWorker pid=28019) action ==  17  New Position 17
(RolloutWorker pid=28020) 42 -> 36 60.0
(RolloutWorker pid=28020) Step() Execution time: 0.271162748336792 seconds
(RolloutWorker pid=28020) action ==  14  New Position 14
(RolloutWorker pid=28019)

In [ ]:
results

Testing of the trainer

In [ ]:
trainer.restore(file_name)
env = gym.make("graphworld-v0")
state = env.reset()

In [ ]:
sum_reward = 0
n_step = 20
for step in range(n_step):
    action = trainer_dqn.compute_action(state)
    state, reward, done, info = env.step(action)
    sum_reward += reward
    #env.render()
    if done == 1:
        print("cumulative reward", sum_reward)
        state = env.reset()
        sum_reward = 0

In [ ]:
ray.shutdown()